# Train a Xgboost Model with Watson Machine Learning 

Notebook created by Zeming Zhao on June, 2021

XGBoost is an implementation of gradient boosted decision trees designed for speed and performance. which is an algorithm that has recently been dominating applied machine learning and Kaggle competitions for structured or tabular data.

This notebook covers the following sections:

1. [Setup Xgboost Model using xgboost lib](#model)<br>

1. [Training the model on CPU with Watson Machine Learning Accelerator](#cpu)<br>

1. [Training the model on GPU with Watson Machine Learning Accelerator](#gpu)<br>

1. [Setup Snap Boosting Model using snapML lib](#snapml-model)<br>

1. [Training the model on CPU with Watson Machine Learning Accelerator](#snapml-cpu)<br>

1. [Training the model on GPU with Watson Machine Learning Accelerator](#snapml-gpu)<br>

## Preparations

### Prepare directory and file for writing Xgboost engine.

In [1]:
from pathlib import Path
model_dir = f'/data/models' 
model_main = f'Xgboost_main.py'
Path(model_dir).mkdir(exist_ok=True)
print("create model directory done.")

create model directory done.


## Step 1 : Setup Xgboost model

<a id = "model"></a>
### create a Xgboost Model based on Xgboost lib 

In [11]:
%%writefile {model_dir}/{model_main}

#from sklearn.datasets import load_iris
#from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, accuracy_score
import numpy as np
import pandas as pd
import datetime
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import xgboost as xgb

# prepare data
# dataset = load_iris()

# X = dataset.data
# y = dataset.target

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Define Parameters for a large regression
n_samples = 2**13 
n_features = 899 
n_info = 600 
data_type = np.float32

# Generate Data using scikit-learn
X,y = make_classification(n_samples=n_samples,
                          n_features=n_features,
                          n_informative=n_info,
                          random_state=123, n_classes=2)

X = pd.DataFrame(X.astype(data_type))
y = pd.Series(y.astype(np.int32))

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2,
                                                    random_state=0)

print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))
print("Number of classes:  %d" % (len(np.unique(y_train))))

D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label=y_test)

# set parameters
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

steps = 20  # The number of training iterations

# setup model and train
start = datetime.datetime.now()
model = xgb.train(param, D_train, steps)
end = datetime.datetime.now()
print ("Xgboost train timecost: %.2gs" % ((end-start).total_seconds()))


# predict
start = datetime.datetime.now()
preds = model.predict(D_test)
end = datetime.datetime.now()
print ("Xgboost predict timecost: %.2gs" % ((end-start).total_seconds()))

# check result
best_preds = np.asarray([np.argmax(line) for line in preds])
print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))

# save the xgboost model into a file
import pickle
filename = './xgboost_model.pkl'
pickle.dump(model, open(filename, 'wb'))              
print("Xgboost model saved successfully.")

Overwriting /data/models/Xgboost_main.py


## Step 2 :  Training the Xgboost model on CPU with Watson Machine Learning Accelerator

### Prepare the model lib for job submission

In [3]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

from matplotlib import pyplot as plt
%pylab inline

import base64
import json
import time
import urllib

Populating the interactive namespace from numpy and matplotlib


### Configuring your environment and project details
To set up your project details, provide your credentials in this cell. You must include your cluster URL, username, and password.

In [4]:
hostname='wmla-console-wmla.apps.wml1x180.ma.platformlab.ibm.com'  # please enter Watson Machine Learning Accelerator host name
# login='username:password' # please enter the login and password
login='admin:p7PMrMMknVQzEb3ptyj0D6XRTO5PQjYL'
es = base64.b64encode(login.encode('utf-8')).decode("utf-8")
# print(es)
commonHeaders={'Authorization': 'Basic '+es}
req = requests.Session()
auth_url = 'https://{}/auth/v1/logon'.format(hostname)
print(auth_url)

a=requests.get(auth_url,headers=commonHeaders, verify=False)
access_token=a.json()['accessToken']
# print("Access_token: ", access_token)

https://wmla-console-wmla.apps.wml1x180.ma.platformlab.ibm.com/auth/v1/logon


In [5]:
dl_rest_url = 'https://{}/platform/rest/deeplearning/v1'.format(hostname)
commonHeaders={'accept': 'application/json', 'X-Auth-Token': access_token}
req = requests.Session()

In [6]:
# Health check
confUrl = 'https://{}/platform/rest/deeplearning/v1/conf'.format(hostname)
r = req.get(confUrl, headers=commonHeaders, verify=False)


### Define the status checking fuction

In [7]:
import tarfile
import tempfile
import os
import json
import pprint
import pandas as pd
from IPython.display import clear_output

def query_job_status(job_id,refresh_rate=3) :

    execURL = dl_rest_url  +'/execs/'+ job_id['id']
    pp = pprint.PrettyPrinter(indent=2)

    keep_running=True
    res=None
    while(keep_running):
        res = req.get(execURL, headers=commonHeaders, verify=False)
        monitoring = pd.DataFrame(res.json(), index=[0])
        pd.set_option('max_colwidth', 120)
        clear_output()
        print("Refreshing every {} seconds".format(refresh_rate))
        display(monitoring)
        pp.pprint(res.json())
        if(res.json()['state'] not in ['PENDING_CRD_SCHEDULER', 'SUBMITTED','RUNNING']) :
            keep_running=False
        time.sleep(refresh_rate)
    return res

### Define the submission commnad

In [8]:
def submit_job_to_wmla (args) :
    starttime = datetime.datetime.now()
    r = requests.post(dl_rest_url+'/execs?args='+args, # files=files,
                  headers=commonHeaders, verify=False)
    if not r.ok:
        print('submit job failed: code=%s, %s'%(r.status_code, r.content))
    job_status = query_job_status(r.json(),refresh_rate=5)
    endtime = datetime.datetime.now()
    print("\nTotallly training cost: ", (endtime - starttime).seconds, " seconds.")

<a id = "cpu"></a>
### Define the submittion parameters using conda env with CPU version XGBoost

In [9]:
# specify the conda env of xgboost and worker device type
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
                     --workerDeviceNum 1 \
                     --workerDeviceType cpu \
                     --conda-env-name rapids-21.06  \
                     --model-main /gpfs/mydatafs/models/' + model_main

print(args)

--exec-start tensorflow --cs-datastore-meta type=fs                      --workerDeviceNum 1                      --workerDeviceType cpu                      --conda-env-name rapids-21.06                       --model-main /gpfs/mydatafs/models/Xgboost_main.py


In [12]:
submit_job_to_wmla (args)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-322,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --...,wmla-322,admin,FINISHED,wmla-322,https://wmla-mss:9080,wmla,/gpfs/myresultfs/admin/batchworkdir/wmla-322/_submitted_code,SingleNodeTensorflowTrain,2021-07-13T08:32:24Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-322',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta '
          'type=fs                      --workerDeviceNum '
          '1                      --workerDeviceType cpu                      '
          '--conda-env-name rapids-21.06                       --model-main '
          '/gpfs/mydatafs/models/Xgboost_main.py ',
  'createTime': '2021-07-13T08:32:24Z',
  'creator': 'admin',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-322',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-322',
  'workDir': '/gpfs/myresultfs/admin/batchworkdir/wmla-322/_submitted_code'}

Totallly training cost:  64  seconds.


## Step 3 :  Training the Xgboost model on GPU with Watson Machine Learning Accelerator

<a id = "gpu"></a>
### Define the submittion parameters using conda env with GPU version XGBoost

In [13]:
# specify the conda env of xgboost and worker device type
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
                     --workerDeviceNum 1 \
                     --workerDeviceType gpu \
                     --conda-env-name dlipy3  \
                     --model-main /gpfs/mydatafs/models/' + model_main

print(args)

--exec-start tensorflow --cs-datastore-meta type=fs                      --workerDeviceNum 1                      --workerDeviceType gpu                      --conda-env-name dlipy3                       --model-main /gpfs/mydatafs/models/Xgboost_main.py


In [14]:
submit_job_to_wmla (args)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-323,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --...,wmla-323,admin,FINISHED,wmla-323,https://wmla-mss:9080,wmla,/gpfs/myresultfs/admin/batchworkdir/wmla-323/_submitted_code,SingleNodeTensorflowTrain,2021-07-13T08:34:48Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-323',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta '
          'type=fs                      --workerDeviceNum '
          '1                      --workerDeviceType gpu                      '
          '--conda-env-name dlipy3                       --model-main '
          '/gpfs/mydatafs/models/Xgboost_main.py ',
  'createTime': '2021-07-13T08:34:48Z',
  'creator': 'admin',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-323',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-323',
  'workDir': '/gpfs/myresultfs/admin/batchworkdir/wmla-323/_submitted_code'}

Totallly training cost:  38  seconds.


## Step 4 :  Setup Snap Boosting model using snapML

<a id = "snapml-model"></a>
### Create a Snap Boosting model based on snapML 

In [15]:
model_main='snapML-'+model_main

In [16]:
%%writefile {model_dir}/{model_main}

#from sklearn.datasets import load_iris
#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, accuracy_score
import numpy as np
import pandas as pd
import datetime
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

#import xgboost as xgb
from snapml import BoostingMachineClassifier as SnapBoostingMachineClassifier


# prepare data
# dataset = load_iris()

# X = dataset.data
# y = dataset.target

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Define Parameters for a large regression
n_samples = 2**13 
n_features = 899 
n_info = 600 
data_type = np.float32

# Generate Data using scikit-learn
X,y = make_classification(n_samples=n_samples,
                          n_features=n_features,
                          n_informative=n_info,
                          random_state=123, n_classes=2)

X = pd.DataFrame(X.astype(data_type))
y = pd.Series(y.astype(np.int32))

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2,
                                                    random_state=0)

print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))
print("Number of classes:  %d" % (len(np.unique(y_train))))

#D_train = xgb.DMatrix(X_train, label=y_train)
#D_test = xgb.DMatrix(X_test, label=y_test)

# set parameters
# param = {
#     'eta': 0.3, 
#     'max_depth': 3,  
#     'objective': 'multi:softprob',  
#     'num_class': 3} 

# steps = 20  # The number of training iterations

# BoostingMachine(objective='mse', num_round=10, min_max_depth=1, 
#   max_max_depth=6, n_threads=1, random_state=None, learning_rate=0.1, 
#   use_histograms=True, hist_nbins=64, use_gpu=False, gpu_id=0, verbose=False, 
#   colsample_bytree=1.0, subsample=1.0, parallel_by_example=False, lambda_l2=0.0)


snap_model = SnapBoostingMachineClassifier(#objective='mse',
                                    #max_max_depth=3,
                                    random_state=42, 
                                    n_jobs=1,
                                    hist_nbins=256,
                                   num_round=20)

# setup model and train
start = datetime.datetime.now()
snap_model.fit(X_train.values, y_train.values)  #.train(param, D_train, steps)
end = datetime.datetime.now()
print ("Xgboost train timecost: %.2gs" % ((end-start).total_seconds()))


# predict
start = datetime.datetime.now()
preds = snap_model.predict(X_test.values)
end = datetime.datetime.now()
print ("Xgboost predict timecost: %.2gs" % ((end-start).total_seconds()))

# check result
best_preds = np.asarray([np.argmax(line) for line in preds])
print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))

# save the xgboost model into a file
import pickle
filename = './snap-xgboost_model.pkl'
pickle.dump(snap_model, open(filename, 'wb'))              
print("Xgboost model saved successfully.")

Overwriting /data/models/snapML-Xgboost_main.py


## Step 5 :  Training the SnapML model on CPU with Watson Machine Learning Accelerator

<a id = "snapml-cpu"></a>
### Re-define the submission parameters

In [17]:
# specify the conda env of snapml and worker device type
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
                     --workerDeviceNum 1 \
                     --workerDeviceType cpu \
                     --conda-env-name snapml-py3.7 \
                     --model-main /gpfs/mydatafs/models/' + model_main
print(args)

--exec-start tensorflow --cs-datastore-meta type=fs                      --workerDeviceNum 1                      --workerDeviceType cpu                      --conda-env-name snapml-py3.7                      --model-main /gpfs/mydatafs/models/snapML-Xgboost_main.py


In [18]:
submit_job_to_wmla (args)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-324,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --...,wmla-324,admin,FINISHED,wmla-324,https://wmla-mss:9080,wmla,/gpfs/myresultfs/admin/batchworkdir/wmla-324/_submitted_code,SingleNodeTensorflowTrain,2021-07-13T08:35:44Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-324',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta '
          'type=fs                      --workerDeviceNum '
          '1                      --workerDeviceType cpu                      '
          '--conda-env-name snapml-py3.7                      --model-main '
          '/gpfs/mydatafs/models/snapML-Xgboost_main.py ',
  'createTime': '2021-07-13T08:35:44Z',
  'creator': 'admin',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-324',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-324',
  'workDir': '/gpfs/myresultfs/admin/batchworkdir/wmla-324/_submitted_code'}

Totallly training cost:  32  seconds.


## Step 6 :  Training the SnapML model on GPU with Watson Machine Learning Accelerator

<a id = "snapml-gpu"></a>
### Re-define the submission parameters

In [19]:
# specify the conda env of snapml and worker device type
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
                     --workerDeviceNum 1 \
                     --workerDeviceType gpu \
                     --conda-env-name snapml-py3.7 \
                     --model-main /gpfs/mydatafs/models/' + model_main
print(args)

--exec-start tensorflow --cs-datastore-meta type=fs                      --workerDeviceNum 1                      --workerDeviceType gpu                      --conda-env-name snapml-py3.7                      --model-main /gpfs/mydatafs/models/snapML-Xgboost_main.py


In [20]:
submit_job_to_wmla (args)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-325,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --...,wmla-325,admin,FINISHED,wmla-325,https://wmla-mss:9080,wmla,/gpfs/myresultfs/admin/batchworkdir/wmla-325/_submitted_code,SingleNodeTensorflowTrain,2021-07-13T08:39:43Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-325',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta '
          'type=fs                      --workerDeviceNum '
          '1                      --workerDeviceType gpu                      '
          '--conda-env-name snapml-py3.7                      --model-main '
          '/gpfs/mydatafs/models/snapML-Xgboost_main.py ',
  'createTime': '2021-07-13T08:39:43Z',
  'creator': 'admin',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-325',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-325',
  'workDir': '/gpfs/myresultfs/admin/batchworkdir/wmla-325/_submitted_code'}

Totallly training cost:  32  seconds.
